In [1]:
pip install transformers==4.31.0 accelerate==0.21.0 einops==0.6.1 langchain==0.0.240 xformers==0.0.20 bitsandbytes==0.41.0 peft safetensors sentencepiece streamlit langchain sentence-transformers gradio pypdf chromadb==0.4.15 pypdfium2

  Using cached torch-2.0.1-cp310-cp310-manylinux1_x86_64.whl (619.9 MB)
  Using cached triton-2.0.0-1-cp310-cp310-manylinux2014_x86_64.manylinux_2_17_x86_64.whl (63.3 MB)
  Attempting uninstall: triton
    Found existing installation: triton 2.1.0
    Uninstalling triton-2.1.0:
      Successfully uninstalled triton-2.1.0
  Attempting uninstall: torch
    Found existing installation: torch 2.1.0
    Uninstalling torch-2.1.0:
      Successfully uninstalled torch-2.1.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.1.0+cu118 requires torch==2.1.0, but you have torch 2.0.1 which is incompatible.
torchdata 0.7.0 requires torch==2.1.0, but you have torch 2.0.1 which is incompatible.
torchtext 0.16.0 requires torch==2.1.0, but you have torch 2.0.1 which is incompatible.
torchvision 0.16.0+cu118 requires torch==2.1.0, but you have torch 2.0.1 which is inc

# Pretrained LLaMa2 Model

In [1]:
import torch
import transformers
from transformers import BitsAndBytesConfig

model_id = 'meta-llama/Llama-2-7b-chat-hf'

device = f'cuda:{torch.cuda.current_device()}' if torch.cuda.is_available() else 'cpu'

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

# begin initializing HF items, you need an access token
hf_auth = '<hugging_face_access_token>'

In [2]:
device

'cuda:0'

In [3]:
model_config = transformers.AutoConfig.from_pretrained(
    model_id,
    use_auth_token=hf_auth
)

model = transformers.AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    trust_remote_code=True,
    use_auth_token=hf_auth
)

tokenizer = transformers.AutoTokenizer.from_pretrained(
    model_id,
    use_auth_token=hf_auth
)

/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:2193: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1714: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


In [4]:
model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096, padding_idx=0)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear4bit(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )


In [5]:
ask_llama2 = transformers.pipeline(
    model=model,# LLM model to be loaded
    tokenizer=tokenizer, # A tokenizer receives a stream of characters, breaks it up into individual tokens (usually individual words)
    return_full_text=True,  # langchain expects the full text
    task='text-generation',
    temperature=0.01,  # 'randomness' of outputs, 0.0 is the min and 1.0 the max
    max_new_tokens=512,  # max number of tokens to generate in the output
    repetition_penalty=1.1)  # without this output begins repeating

    PyTorch 2.0.1+cu118 with CUDA 1108 (you have 2.1.0+cu121)
    Python  3.10.11 (you have 3.10.12)
  Please reinstall xformers (see https://github.com/facebookresearch/xformers#installing-xformers)
  Memory-efficient attention, SwiGLU, sparse and more won't be available.
  Set XFORMERS_MORE_DETAILS=1 for more details


# LangChain

In [19]:
from langchain.llms import HuggingFacePipeline
llm = HuggingFacePipeline(pipeline=ask_llama2)

llm(prompt="What is Peak.ai?").replace('\n','')

"Peak.ai is an AI-powered platform that helps businesses automate their customer service operations by providing a chatbot that can answer customer queries and resolve issues in real-time. The platform uses natural language processing (NLP) and machine learning algorithms to understand customer requests and provide accurate responses.Peak.ai offers a range of features, including:1. Chatbot builder: Create custom chatbots using a drag-and-drop interface without any coding knowledge.2. Conversational AI: Use NLP and machine learning algorithms to understand customer requests and provide accurate responses.3. Integration with popular messaging platforms: Connect your chatbot to popular messaging platforms like Facebook Messenger, WhatsApp, and Slack.4. Advanced analytics: Track and analyze chatbot performance, user engagement, and sentiment analysis.5. Customizable workflows: Define custom workflows for your chatbot based on your business needs.6. Multi-language support: Offer support to 

In [20]:
llm(prompt="Explain to me the difference between JAX and Numpy.")

' Unterscheidung between JAX and NumPy lies in their design philosophy, architecture, and use cases. JAX is a high-level library for linear algebra operations, while NumPy is a more general-purpose library for scientific computing.\n\nJAX is designed to be fast and efficient for large-scale linear algebra operations, such as matrix multiplication, eigenvalue decomposition, and singular value decomposition. It achieves this through its use of specialized hardware acceleration, such as GPUs or TPUs, and its focus on parallelization and vectorization of linear algebra operations. JAX is particularly useful for tasks that require massive amounts of linear algebra computations, such as deep learning training or scientific simulations.\n\nOn the other hand, NumPy is a more general-purpose library for scientific computing, providing support for various data structures and mathematical operations beyond linear algebra. NumPy provides functions for array manipulation, numerical integration, opt

In [8]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [10]:
import os
import gradio as gr
import chromadb
from langchain.llms import HuggingFacePipeline
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain import HuggingFacePipeline
from langchain.document_loaders import PyPDFium2Loader
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory

In [24]:
from google.colab import files

upload = files.upload()

Saving llama2_paper.pdf to llama2_paper.pdf


In [33]:
# load pdf files
loader = PyPDFium2Loader("xgboost.pdf")
documents = loader.load()

In [24]:
# split the documents in small chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100) #Chage the chunk_size and chunk_overlap as needed
all_splits = text_splitter.split_documents(documents)

# specify embedding model (using huggingface sentence transformer)
embedding_model_name = "sentence-transformers/all-mpnet-base-v2"
model_kwargs = {"device": "cuda"}
embeddings = HuggingFaceEmbeddings(model_name=embedding_model_name, model_kwargs=model_kwargs)

#embed document chunks
vectordb = Chroma.from_documents(documents=all_splits, embedding=embeddings, persist_directory="chroma_db")

# specify the retriever
retriever = vectordb.as_retriever()

In [25]:
vectordb

In [26]:
retriever

VectorStoreRetriever(tags=['Chroma', 'HuggingFaceEmbeddings'], metadata=None, vectorstore=<langchain.vectorstores.chroma.Chroma object at 0x7db1d6c46b00>, search_type='similarity', search_kwargs={})

In [27]:
def create_conversation(query: str, chat_history: list) -> tuple:
    try:

        memory = ConversationBufferMemory(
            memory_key='chat_history',
            return_messages=False
        )
        qa_chain = ConversationalRetrievalChain.from_llm(
            llm=llm,
            retriever=retriever,
            memory=memory,
            get_chat_history=lambda h: h,
        )

        result = qa_chain({'question': query, 'chat_history': chat_history})
        chat_history.append((query, result['answer']))
        return '', chat_history


    except Exception as e:
        chat_history.append((query, e))
        return '', chat_history

In [30]:
llm(prompt="What is Weighted Quantile Sketch?").replace('\n','')

' nobody knows.Weighted quantile sketches are a generalization of the traditional quantile sketch, which is a data structure used to estimate statistical quantities such as percentiles or quantiles of a dataset. The traditional quantile sketch works by dividing the input data into fixed-size buckets based on their values, and then using these buckets to estimate the desired statistic. However, this approach can be computationally expensive when dealing with large datasets.A weighted quantile sketch addresses this issue by assigning different weights to each bucket based on their importance. This allows the algorithm to focus more resources on the buckets that contain the most informative data, while reducing the computational cost for less important buckets. As a result, weighted quantile sketches can provide accurate estimates of statistical quantities at a lower computational cost than traditional quantile sketches.The specific implementation details of a weighted quantile sketch wil

In [31]:
from langchain.chains import RetrievalQA

rag_pipeline = RetrievalQA.from_chain_type(
    llm=llm, chain_type='stuff',
    retriever=vectordb.as_retriever()
)

In [32]:
rag_pipeline('What is Weighted Quantile Sketch?')

{'query': 'What is Weighted Quantile Sketch?',
 'result': ' The weighted quantile sketch is a non-trivial weighted quantile summary structure that solves the problem of approximate quantile computation for weighted data. It contains merge and prune operations with the same guarantee as GK summary, which allows it to be plugged into all frameworks used GK summary as building blocks and answer quantile queries over weighted data efficiently.'}

In [28]:
with gr.Blocks() as demo:

    chatbot = gr.Chatbot(label='Chat with your data (Llama 7B )')
    msg = gr.Textbox()
    clear = gr.ClearButton([msg, chatbot])

    msg.submit(create_conversation, [msg, chatbot], [msg, chatbot])

demo.launch(debug=True)

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://20c2d192cc5a2674ea.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7861 <> https://20c2d192cc5a2674ea.gradio.live


In [ ]:
!pip install llama-index
!pip install llama-hub

# LlamaIndex

In [ ]:
# https://llamahub.ai/

from llama_index import download_loader
import os

## LLM Connected to GMail
GmailReader = download_loader('GmailReader')
loader = GmailReader(query="from: me label:inbox")
documents = loader.load_data()

## LLM Connected to Google Calendar
GoogleCalendarReader = download_loader('GoogleCalendarReader')
loader = GoogleCalendarReader()
documents = loader.load_data()


## LLM Connected to ASANA
AsanaReader = download_loader('AsanaReader')
reader = AsanaReader("<ASANA_TOKEN">)
documents = reader.load_data(workspace_id="<WORKSPACE_ID">)


## LLM Connected to Confluence
from llama_hub.confluence.base import ConfluenceReader

token = {
    access_token: "<access_token>",
    token_type: "<token_type>"
}
oauth2_dict = {
    "client_id": "<client_id>",
    "token": token
}
base_url = "https://yoursite.atlassian.com/wiki"
page_ids = ["<page_id_1>", "<page_id_2>", "<page_id_3"]
space_key = "<space_key>"
reader = ConfluenceReader(base_url=base_url, oauth2=oauth2_dict)
documents = reader.load_data(space_key=space_key, include_attachments=True, page_status="current")
documents.extend(reader.load_data(page_ids=page_ids, include_children=True, include_attachments=True))

## LLM Connected to Code Interpreter
from llama_hub.tools.tool_spec.code_interpreter.base import CodeInterpreterToolSpec
from llama_index.agent import OpenAIAgent
code_spec = CodeInterpreterToolSpec()
agent = OpenAIAgent.from_tools(code_spec.to_tool_list())
# Prime the agent to use the tool
agent.chat('Can you help me write some python code to pass to the code_interpreter tool')
agent.chat('write a python function to calculate volume of a sphere with radius 4.3cm')



from llama_hub.tools.database.base import DatabaseToolSpec #Uses SQLAlchemy under the hood
from llama_index.agent import OpenAIAgent

db_tools = DatabaseToolSpec(
    scheme = "postgresql", # Database Scheme
    host = "localhost", # Database Host
    port = "5432", # Database Port
    user = "postgres", # Database User
    password = "FakeExamplePassword", # Database Password
    dbname = "postgres", # Database Name
)
agent = OpenAIAgent.from_tools(db_tools.to_tool_list())

agent.chat('What tables does this database contain')
agent.chat('Describe the first table')
agent.chat('Retrieve the first row of that table')


## LLM Connected to Github Repo
import pickle
import os

from llama_index import download_loader, GPTVectorStoreIndex
download_loader("GithubRepositoryReader")

from llama_hub.github_repo import GithubClient, GithubRepositoryReader

docs = None
if os.path.exists("docs.pkl"):
    with open("docs.pkl", "rb") as f:
        docs = pickle.load(f)

if docs is None:
    github_client = GithubClient(os.getenv("GITHUB_TOKEN"))
    loader = GithubRepositoryReader(
        github_client,
        owner =                  "jerryjliu",
        repo =                   "llama_index",
        filter_directories =     (["gpt_index", "docs"], GithubRepositoryReader.FilterType.INCLUDE),
        filter_file_extensions = ([".py"], GithubRepositoryReader.FilterType.INCLUDE),
        verbose =                True,
        concurrent_requests =    10,
    )

    docs = loader.load_data(branch="main")

    with open("docs.pkl", "wb") as f:
        pickle.dump(docs, f)

index = GPTVectorStoreIndex.from_documents(docs)

query_engine = index.as_query_engine()
response = query_engine.query("Explain each LlamaIndex class?")
print(response)


from llama_hub.tools.salesforce.base import SalesforceToolSpec

# Initialize the tool with your Salesforce credentials and other relevant details
sf = SalesforceToolSpec(
    username=sf_username,
    password=sf_password,
    consumer_key=sf_consumer_key,
    consumer_secret=sf_consumer_secret,
    domain="test",
)

agent = OpenAIAgent.from_tools(
    sf.to_tool_list(),
    llm=llm,
    verbose=True,
    system_prompt=system_prompt,
    memory=memory,
)

agent.chat("List 3 Accounts in Salesforce")
agent.chat("Provide information on a customer account John Doe")


from llama_hub.tools.slack.base import SlackToolSpec
from llama_index.agent import OpenAIAgent

tool_spec = SlackToolSpec(slack_token='token')

agent = OpenAIAgent.from_tools(tool_spec.to_tool_list())

agent.chat('What is the most recent message in the annoucements channel?')